<a href="https://colab.research.google.com/github/iamgagan/AI_Resume/blob/main/AI_Resume_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
from typing import Dict, Any, Union
from dotenv import load_dotenv
from crewai import Agent, Task, Crew, Process
from crewai.crews.crew_output import CrewOutput
from langchain_openai import ChatOpenAI
from langchain.tools import tool
import requests
from langchain_community.document_loaders import PyMuPDFLoader

# Load environment variables
load_dotenv()

api_key = api_key
model = ChatOpenAI(api_key=api_key, model_name="gpt-4-turbo", temperature=0.8)

@tool
def fetch_pdf_content(pdf_path: str) -> str:
    """Reads a local PDF and returns the content"""
    try:
        loader = PyMuPDFLoader(pdf_path)
        data = loader.load()[0]
        return data.page_content
    except Exception as e:
        return f"Error reading PDF: {str(e)}"

@tool
def get_webpage_contents(url: str) -> str:
    """Reads the webpage with a given URL and returns the page content"""
    try:
        response = requests.get(url, verify=False)  # Disable SSL verification
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        return f"Error fetching webpage: {str(e)}"

job_crawler = Agent(
    role='Job Description Crawler',
    goal='Extract the relevant job description, requirements and qualifications',
    backstory='Specialized in parsing HTML and retrieving important information from it',
    verbose=True,
    tools=[get_webpage_contents],
    allow_delegation=False,
    llm=model
)

cv_modifier = Agent(
    role='CV/Resume Writer',
    goal='Write a top-notch CV that increases the chance of landing an interview',
    backstory='Expert in writing CV that is best received by the recruiting team and HR',
    verbose=True,
    tools=[fetch_pdf_content],
    allow_delegation=False,
    llm=model
)

recruiter = Agent(
    role='Hiring Manager',
    goal='Analyze how well a candidate is suited for a job description, given their CV',
    backstory='Experienced hiring manager with an specialization of giving feedback to job seekers',
    verbose=True,
    allow_delegation=False,
    llm=model
)

def extract_job_information(page_url: str) -> Task:
    return Task(
        description=f"""
        Given this url: {page_url}, extract the job description, and relative information about the job.
        Return the information as a JSON string with the following structure:
        {{
            "title": "Job Title",
            "company": "Company Name",
            "description": "Full job description",
            "requirements": ["Requirement 1", "Requirement 2", ...],
            "qualifications": ["Qualification 1", "Qualification 2", ...]
        }}
        """,
        agent=job_crawler,
        expected_output="JSON string containing job information",
    )

def cv_modifying(cv_path: str, job_info: str) -> Task:
    return Task(
        description=f"""
        Read the CV at this local path: {cv_path}, then modify the keypoints and the order of the skills,
        to make it emphasize what is needed by the job. Do NOT add any extra skill or new information, keep it honest.
        Use the following job information: {job_info}
        Return the modified CV as a JSON string with the following structure:
        {{
            "educational_background": ["Education 1", "Education 2", ...],
            "work_experience": ["Experience 1", "Experience 2", ...],
            "skills": ["Skill 1", "Skill 2", ...],
            "achievements": ["Achievement 1", "Achievement 2", ...]
        }}
        """,
        agent=cv_modifier,
        expected_output="JSON string containing modified CV",
    )

def evaluate_task(job_info: str, modified_cv: str) -> Task:
    return Task(
        description=f"""
        Given the job information: {job_info}
        And the modified CV: {modified_cv}
        Evaluate the candidate's suitability for the job.
        Return the evaluation as a JSON string with the following structure:
        {{
            "score": 85,
            "strengths": ["Strength 1", "Strength 2", ...],
            "weaknesses": ["Weakness 1", "Weakness 2", ...],
            "overall_assessment": "Brief overall assessment"
        }}
        """,
        agent=recruiter,
        expected_output="JSON string containing evaluation results",
    )

def parse_crew_output(output: Union[str, CrewOutput, Dict[str, Any]]) -> Dict[str, Any]:
    if isinstance(output, CrewOutput):
        # Extract the relevant information from CrewOutput
        return parse_json_output(str(output))
    elif isinstance(output, str):
        return parse_json_output(output)
    elif isinstance(output, dict):
        return output
    else:
        return {"error": f"Unexpected output type: {type(output)}"}

def parse_json_output(output: str) -> Dict[str, Any]:
    try:
        # Try to extract JSON from the string
        json_start = output.index('{')
        json_end = output.rindex('}') + 1
        json_str = output[json_start:json_end]
        return json.loads(json_str)
    except (ValueError, json.JSONDecodeError):
        return {"error": f"Failed to parse JSON from string: {output}"}

def process_resume(job_url: str, cv_path: str) -> Dict[str, Any]:
    try:
        extract_job_information_task = extract_job_information(job_url)
        crew = Crew(
            agents=[job_crawler, cv_modifier, recruiter],
            tasks=[extract_job_information_task],
            verbose=2
        )
        job_info = crew.kickoff()
        job_info_parsed = parse_crew_output(job_info)

        cv_modifying_task = cv_modifying(cv_path, json.dumps(job_info_parsed))
        crew = Crew(
            agents=[job_crawler, cv_modifier, recruiter],
            tasks=[cv_modifying_task],
            verbose=2
        )
        modified_cv = crew.kickoff()
        modified_cv_parsed = parse_crew_output(modified_cv)

        evaluate = evaluate_task(json.dumps(job_info_parsed), json.dumps(modified_cv_parsed))
        crew = Crew(
            agents=[job_crawler, cv_modifier, recruiter],
            tasks=[evaluate],
            verbose=2
        )
        evaluation = crew.kickoff()
        evaluation_parsed = parse_crew_output(evaluation)

        return {
            "job_info": job_info_parsed,
            "modified_cv": modified_cv_parsed,
            "evaluation": evaluation_parsed
        }
    except Exception as e:
        print(f"Error in process_resume: {e}")
        return {"error": str(e)}

def main():
    cv_path = r'/content/Resume1.pdf'
    job_url = "https://jobs.silkroad.com/FirstCommand/Careers/jobs/2642?source=LinkedIn"

    result = process_resume(job_url, cv_path)

    if "error" in result:
        print(f"An error occurred: {result['error']}")
    else:
        print("Job Information:")
        print(json.dumps(result["job_info"], indent=2))
        print("\nModified CV:")
        print(json.dumps(result["modified_cv"], indent=2))
        print("\nEvaluation:")
        print(json.dumps(result["evaluation"], indent=2))

if __name__ == "__main__":
    main()